# 赛题 [填写赛题编号，如: E题]：[填写赛题名称，如: 交通流量管控]

> **团队成员**: 👨‍💻 @俊宇 (建模与编程), 📊 @(数据分析), ✍️ @队友B (论文写作)
> **文件版本**: V0.1 - 初始数据探索
> **最后更新**: [请在此处填写日期和时间]

---
### **核心思路与模型框架**
*在这个区域，用几句话初步描述你们团队讨论出的解题大方向*
1. **问题一**: 通过轨迹匹配推断转向流量，并用K-Means进行时段划分。
2. **问题二**: 建立排队论模型，利用遗传算法优化信号灯配时方案。
3. ...
---

In [1]:
# ===================================================================
# 模块一：全局初始化 (Global Initialization)
# ===================================================================

# 导入所有可能用到的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import warnings
import pulp
# ... import other libraries as needed

# 全局美化设置
plt.rcParams['font.sans-serif'] = ['SimHei'] # 解决中文显示问题
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
sns.set_style('whitegrid') # 设置图表风格
warnings.filterwarnings("ignore", category=UserWarning)
print("✅ 指挥中心已启动，所有库准备就绪！(警告过滤器已开启)")

✅ 指挥中心已启动，所有库准备就绪！(警告过滤器已开启)


# ===================================================================
# 模块二：数据加载与预处理 (Data Loading & Preprocessing)
# 负责人: @队友A
# ===================================================================

### 2.1 数据加载
* **任务**: 加载题目附件中的所有数据文件。

# 🗺️ Step 1: 数据收集 (Data Collection)
> **🎯 目标**: 加载题目附件中的所有数据文件，**确认数据已成功读入**。
> **🕵️‍♂️ 你的任务**: 
> 1. 将所有附件文件和本Notebook放在**同一个文件夹**下。
> 2. 在下方代码单元格中，**修改`file_path_x`变量为正确的文件名**。
> 3. 按 **Shift+Enter** 执行代码。

In [2]:
# ----------------- 请在这里修改您的文件名 -----------------
file_path_1 = r'E:\我的大学四年\大一下\数模\23年华数杯C题\附件.xlsx'
#file_path_2 = '文件2.csv'
# file_path_3 = '...' # 如果有更多文件，在此处添加
# --------------------------------------------------------

try:
    # 尝试用Pandas读取数据，您的主要竞赛库是pd.read_excel()和pd.read_csv()
    df1 = pd.read_excel(file_path_1)
    #df2 = pd.read_csv(file_path_2, encoding='gbk') # CSV文件有时需要指定编码，如gbk
    # df3 = ...
    print(f"✔️ 所有附件数据加载成功！")
    
    print("\n[--df1数据预览 (前5行)--]")
    display(df1.head())
    #print("\n[--df2数据预览 (前5行)--]")
    #display(df2.head())
    
except FileNotFoundError as e:
    print(f"❌ 文件未找到，请仔细检查文件名和路径是否完全正确: {e}")
except Exception as e:
    print(f"❌ 数据加载失败，可能是编码问题或其他错误: {e}")

✔️ 所有附件数据加载成功！

[--df1数据预览 (前5行)--]


,编号,母亲年龄,婚姻状况,教育程度,妊娠时间（周数）,分娩方式,CBTS,EPDS,HADS,婴儿行为特征,...,整晚睡眠时间（时：分：秒）,睡醒次数,入睡方式,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1.0,34.0,已婚,研究生,37.0,自然分娩,3.0,13.0,9.0,中等型,...,10:00:00,3.0,抚触法,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,33.0,已婚,研究生,42.0,自然分娩,0.0,0.0,3.0,安静型,...,11:00:00,0.0,环境营造法,NaN,补充说明（数值含义）,NaN,NaN,NaN,NaN,NaN
2,3.0,37.0,已婚,研究生,41.0,自然分娩,4.0,8.0,9.0,安静型,...,12:00:00,1.0,抚触法,NaN,数值,婚姻状况,教育程度,分娩方式,婴儿性别,入睡方式
3,4.0,31.0,已婚,研究生,37.5,自然分娩,6.0,16.0,13.0,安静型,...,11:00:00,2.0,哄睡法,NaN,1,未婚,小学,自然分娩,男性,哄睡法：通过轻声哄唤、摇晃、拍打等方式，使婴儿感到安全和舒适，进而入睡。
4,5.0,36.0,未婚,研究生,40.0,自然分娩,1.0,3.0,3.0,中等型,...,10:30:00,1.0,环境营造法,NaN,2,已婚,初中,剖宫产,女性,抚触法：通过轻柔的按摩、揉捏等方式，促进婴儿的放松和入睡。


# 🧭 Step 2: 数据探索 (Exploratory Data Analysis - EDA)
> **🎯 目标**: 快速摸清每个数据集的“家底”（**结构、分布、缺失情况**），对数据建立宏观认知。
> **🕵️‍♂️ 你的任务**:
> 1. 逐个执行下方的代码单元格。
> 2. **仔细观察**每个单元格的输出结果。
> 3. 将你的**所有发现和疑惑**，都记录在最后的 **“🕵️‍♂️ 你的【侦察笔记】”** 区域。

In [3]:
# --------------------------------------------------------------------------------------------------------------------------------
# [-- 2.1 检查df1整体信息 --]
# --------------------------------------------------------------------------------------------------------------------------------
print(f"[-- df1 数据整体概览 (info) --]")
df1.info()
print(f"**" * 40 + "\n")

print(f"[-- df1 数值型数据统计概览 (describe) --]")
display(df1.describe())
print(f"**" * 40 + "\n")

print(f"[-- df1 非数值型数据统计概览 (describe for objects) --]")
display(df1.describe(include='object'))
print(f"**" * 40 + "\n")

[-- df1 数据整体概览 (info) --]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   编号             401 non-null    float64
 1   母亲年龄           401 non-null    float64
 2   婚姻状况           443 non-null    object 
 3   教育程度           401 non-null    object 
 4   妊娠时间（周数）       401 non-null    float64
 5   分娩方式           401 non-null    object 
 6   CBTS           401 non-null    float64
 7   EPDS           401 non-null    float64
 8   HADS           401 non-null    float64
 9   婴儿行为特征         381 non-null    object 
 10  婴儿性别           401 non-null    object 
 11  婴儿年龄（月）        401 non-null    float64
 12  整晚睡眠时间（时：分：秒）  381 non-null    object 
 13  睡醒次数           381 non-null    float64
 14  入睡方式           381 non-null    object 
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    16 non-null     object 
 17  Unnamed: 17    7 non-null   

,编号,母亲年龄,妊娠时间（周数）,CBTS,EPDS,HADS,婴儿年龄（月）,睡醒次数,Unnamed: 15
count,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,401.000000,381.000000,0.0
mean,205.201995,30.154613,39.142394,5.965087,9.082294,7.865337,1.952618,1.467192,NaN
std,118.814989,4.363603,1.885404,4.975819,6.716823,4.275725,0.821736,1.621273,NaN
min,1.000000,19.000000,26.500000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN
25%,103.000000,27.000000,38.000000,2.000000,4.000000,5.000000,1.000000,0.000000,NaN
50%,205.000000,30.000000,39.200000,5.000000,8.000000,7.000000,2.000000,1.000000,NaN
75%,309.000000,33.000000,40.400000,9.000000,13.000000,11.000000,3.000000,2.000000,NaN
max,410.000000,47.000000,43.000000,21.000000,28.000000,20.000000,3.000000,10.000000,NaN


********************************************************************************

[-- df1 非数值型数据统计概览 (describe for objects) --]


,婚姻状况,教育程度,分娩方式,婴儿行为特征,婴儿性别,整晚睡眠时间（时：分：秒）,入睡方式,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
count,443,401,401,381,401,381,381,16,7,6,3,3,6
unique,2,5,2,3,2,15,5,16,7,6,3,3,6
top,已婚,研究生,自然分娩,中等型,男性,10:00:00,环境营造法,补充说明（数值含义）,婚姻状况,教育程度,分娩方式,婴儿性别,入睡方式
freq,429,187,396,221,207,107,164,1,1,1,1,1,1


********************************************************************************



你的【侦察笔记】:
你的任务: 这是一个真正的数据集！走完下一步，在这里写下你从上面所有输出所得到的初步发现、结论和问题，这对后面的建模至关重要！

关于df1：

数据量与缺失值: df1共有443行，但许多列的非空值只有401行或381行，存在大量缺失值。此外，最后几行（从第402行开始）似乎是补充说明，而非有效数据，需要清理。

列名与脏乱列: 列名中存在不规范的字符，如妊娠时间（周数）和...。同时，Unnamed: 15到Unnamed: 21这些列包含极少或没有数据，应予删除。

数据类型与格式问题: 整晚睡眠时间（时：分：秒）列的数据类型是object，但其内容是时间，需要转换成统一的时间格式（如timedelta）以便计算。

异常值或不一致数据: describe结果显示婴儿年龄（月）的max值为3.00，但数据预览中出现过4.0，可能存在数据不一致。婴儿行为特征中存在中等型...的格式，也需要处理。

分类数据: 婚姻状况、教育程度、分娩方式等列为object类型，其内容需要进行进一步检查，以确保值是唯一的且没有拼写错误。

侦察结论/概况：

数据表df1包含了母亲和婴儿的多维度信息，但存在明显的脏数据、缺失值和格式不一致问题，需要进行彻底的数据清洗才能用于后续分析和建模。特别是补充说明行和Unnamed列需要先行删除。

df1的列名需要统一规范化，便于代码调用。

数据中的缺失值需要根据各列的特点选择合适的填充策略。

核心列整晚睡眠时间的格式需要转换。

# 🧹 Step 3: 数据清洗 (Data Cleaning)
> **🎯 目标**: 处理我们在探索阶段发现的**缺失值、异常值、重复值和错误数据类型**。
> **🕵️‍♂️ 你的任务**: 根据你的【侦察笔记】，在下方代码块中，“对症下药”，编写代码清洗数据，并**在【清洗日志】中记录你的每一步操作和理由**。

In [4]:
import pandas as pd
import numpy as np
import re

df_cleaned = df1.copy()

print("--- 开始数据清洗 ---")

# 步骤 1：处理多余行和列
# 从 info() 结果看，最后几行是补充说明，而非数据，需要删除。
# 另外，Unnamed 系列列几乎为空，也需要删除。
df_cleaned = df_cleaned.iloc[0:401, :]
df_cleaned = df_cleaned.drop(columns=['Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'])
print("⚫ 步骤 1: 已删除多余行和列。")
print(f"⚫ 剩余数据量: {df_cleaned.shape}")

# 步骤 2: 处理列名不规范
# 移除列名中的特殊字符和多余空格，便于后续操作。
df_cleaned.columns = df_cleaned.columns.str.replace('（时：分：秒）', '', regex=False).str.replace('...', '', regex=False).str.strip()
print("⚫ 步骤 2: 列名已清洗。")
print(f"⚫ 新列名: {df_cleaned.columns.tolist()}")

# 步骤 3: 处理时间数据格式
# 使用自定义函数处理不规范的时间字符串，将其转换为pd.to_timedelta可接受的格式。
def parse_time_string(time_str):
    if pd.isna(time_str):
        return np.nan
    time_str = str(time_str).strip()
    match = re.match(r'(\d+):(\d+):(\d+)', time_str)
    if match:
        return time_str
    else:
        # 如果不是标准格式，尝试补全为标准格式，例如 '10' -> '10:00:00'
        parts = time_str.split(':')
        if len(parts) == 1 and parts[0].isdigit():
            return f"{int(parts[0]):02d}:00:00"
        elif len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            return f"{int(parts[0]):02d}:{int(parts[1]):02d}:00"
    return time_str

df_cleaned['整晚睡眠时间'] = df_cleaned['整晚睡眠时间'].astype(str).apply(parse_time_string)
df_cleaned['整晚睡眠时间'] = pd.to_timedelta(df_cleaned['整晚睡眠时间'])
print("⚫ 步骤 3: 已将'整晚睡眠时间'列转换为时间格式。")

# 步骤 4: 处理缺失值
# 考虑到数据量较少，对不同列采用不同的缺失值填充策略。
# 采用中位数对数值型数据进行填充，众数对分类数据进行填充。
numeric_cols = df_cleaned.select_dtypes(include='float64').columns
for col in numeric_cols:
    median_val = df_cleaned[col].median()
    df_cleaned[col].fillna(median_val, inplace=True)

categorical_cols = df_cleaned.select_dtypes(include='object').columns
for col in categorical_cols:
    mode_val = df_cleaned[col].mode()[0]
    df_cleaned[col].fillna(mode_val, inplace=True)

print("⚫ 步骤 4: 缺失值已处理。")
print("⚫ 缺失值处理后，各列非空值数量：")
df_cleaned.info()


print("\n--- 数据清洗结束 ---")
print("✔️ 已成功根据你的【侦察笔记】清洗数据！")

--- 开始数据清洗 ---
⚫ 步骤 1: 已删除多余行和列。
⚫ 剩余数据量: (401, 15)
⚫ 步骤 2: 列名已清洗。
⚫ 新列名: ['编号', '母亲年龄', '婚姻状况', '教育程度', '妊娠时间（周数）', '分娩方式', 'CBTS', 'EPDS', 'HADS', '婴儿行为特征', '婴儿性别', '婴儿年龄（月）', '整晚睡眠时间', '睡醒次数', '入睡方式']
⚫ 步骤 3: 已将'整晚睡眠时间'列转换为时间格式。
⚫ 步骤 4: 缺失值已处理。
⚫ 缺失值处理后，各列非空值数量：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   编号        401 non-null    float64        
 1   母亲年龄      401 non-null    float64        
 2   婚姻状况      401 non-null    object         
 3   教育程度      401 non-null    object         
 4   妊娠时间（周数）  401 non-null    float64        
 5   分娩方式      401 non-null    object         
 6   CBTS      401 non-null    float64        
 7   EPDS      401 non-null    float64        
 8   HADS      401 non-null    float64        
 9   婴儿行为特征    401 non-null    object         
 10  婴儿性别      401 non-null    object         
 11  婴儿年龄（月）   401

C:\Users\crj\AppData\Local\Temp\ipykernel_13760\4118045121.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(median_val, inplace=True)
C:\Users\crj\AppData\Local\Temp\ipykernel_13760\4118045121.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

你的【清洗日志】:
你的任务: 在这里详细记录你的每一步清洗操作和选择该操作的理由。这是论文中"数据预处理"章节的核心素材！

数据行与列处理:

操作: 删除Unnamed: 15到Unnamed: 21等空值较多的列。

理由: 在数据探索阶段，我们发现这些列几乎为空，且部分行包含的是数据补充说明，而非有效数据。删除这些脏乱的列和行，可以确保数据表的纯净性和结构规范。

列名清洗:

操作: 将妊娠时间（周数）等列名中的中文括号和特殊字符...移除，并去除多余空格。

理由: 规范化的列名更便于在代码中调用，避免因特殊字符导致语法错误，提高代码的可读性和可维护性。

数据类型转换:

操作: 将整晚睡眠时间列从object类型转换为timedelta64[ns]类型。

理由: 在数据探索阶段发现此列虽然是时间数据，但类型不正确。转换为timedelta格式后，我们可以对睡眠时间进行直接的数学运算和统计分析，从而更准确地研究婴儿的睡眠模式。

缺失值填充:

操作: 对所有数值型列（如母亲年龄、睡醒次数等）使用中位数进行填充；对所有类别型列（如婚姻状况、教育程度等）使用众数进行填充。

理由: 中位数对异常值更具鲁棒性，能够更好地代表数值型数据的集中趋势，特别是在数据分布偏斜时。众数填充是处理分类数据缺失值的常用方法，可以保留原数据的分布特征。



# 🖇️ Step 4: 数据集成 (Data Integration)
> **🎯 目标**: 将多个清洗干净的数据集合并(merge)成一个强大的“**主分析数据集**”。
> **🕵️‍♂️ 你的任务**: 找到合适的关联键(key)，使用`pd.merge()`将df1_cleaned和df2_cleaned合并。

In [ ]:
try:
    # --- 🔴 请在这里修改关联键'key_column_name' ---
    # key_column_name = '商品ID' # 假设这是共同的列名
    # master_df = pd.merge(df2_cleaned, df1_cleaned, on=key_column_name, how='left')
    
    # print("✅ 数据集成功合并！预览合并后的主数据集：")
    # display(master_df.head())
    # print("\n--- 合并后主数据集信息 ---")
    # master_df.info()
    print("🖇️ (占位符) 请在此处编写你的数据集合并代码")
    
except KeyError as e:
    print(f"❌ 合并失败，请检查两个数据表中的关联列名是否完全一致: {e}")

# 🎨 Step 5 (Bonus): 初步可视化洞察
> **🎯 目标**: 对整合后的干净数据，进行初步的可视化，寻找规律。
> **🕵️‍♂️ 你的任务**: 尝试使用`sns.lineplot`, `sns.barplot`等，画1-2张你认为最有价值的图。

In [ ]:
# 示例：分析每日的总销售额趋势
# daily_revenue = master_df.groupby('日期')['销售额'].sum()
# plt.figure(figsize=(15, 6))
# sns.lineplot(data=daily_revenue)
# plt.title("每日总销售额趋势图", fontsize=16)
# plt.show()
print("🎨 (占位符) 请在此处尝试你的第一个可视化分析！")

---
### 🎉 恭喜！你已完成数据侦察与清洗的核心任务！🎉
> **下一步行动**:
> 1. 将这份Notebook文件保存，并命名为`[赛题编号]_Data_Report_V1.ipynb`。
> 2. **在Notion的建模日志中，简要汇报你的核心发现**。
> 3. **通知 @俊宇 (我) 进行Code Review，并讨论后续的建模方向**！
---


# ===================================================================
# --- [阶段性交付点] --- 数据分析师完成工作 ---
# ===================================================================

# ===================================================================
# 模块三：模型构建与求解 (Modeling & Solving)
# 负责人: @俊宇
# ===================================================================

In [ ]:
**(Code) 代码单元格**
```python
# ... 在这里放置你问题一的模型代码 ...
# kmeans = KMeans(n_clusters=3, random_state=42)
# ...```
**(M) Markdown 单元格**
```markdown
### 3.2 [问题二的模型，如：信号灯优化模型]
* **模型思路**: 建立一个以总通行时间最短为目标的线性规划模型...```
**(Code) 代码单元格**
```python
# ... 在这里放置你问题二的模型代码 ...
# prob = pulp.LpProblem(...)
# ...

# ===================================================================
# 模块四：结果分析与可视化 (Result Analysis)
# 负责人: @俊宇 & @队友B
# ===================================================================

### 4.1 核心结果汇总
* **任务**: 将模型求解出的关键结果，以清晰的表格和图表形式展示出来，并存为图片或文件，方便论文引用。
* **(重要: 这里的结果，要和Notion指挥中心的“最终结果汇总区”保持绝对一致！)**

In [ ]:
# ... 在这里放置所有用于生成最终结果表格和图表的代码 ...
# final_result_df.to_excel("问题一结果.xlsx")
# plt.savefig("核心结果图.png", dpi=300)